In [1]:
import numpy as np
import math

#L is the size of total words in the vocab like my name is fahad that is 4 and d_k and d_v is same as d_total
L, d_k, d_v = 4, 8, 8
q = np.random.randn(L, d_k)
k = np.random.randn(L, d_k)
v = np.random.randn(L, d_v)

In [2]:
print("Q\n", q)
print("K\n", k)
print("V\n", v.shape)

Q
 [[ 1.09055468 -2.0516344   0.91705551 -0.80064762 -0.70158921 -0.8760401
  -0.1307713   2.34696991]
 [ 0.43011261  0.45461376  0.45879596  0.54999023 -0.53638167 -0.25319405
   0.24376715 -0.89314806]
 [ 0.45241956  2.64528065  0.50091089 -0.66364911 -0.19422367  0.72443613
   0.95165321 -0.68233126]
 [-0.65959933 -1.69094675  0.14901582  0.56326842 -0.45650084  0.2401893
  -0.5038496  -0.17789481]]
K
 [[-0.12870307  0.76705569  0.34871384  0.39214776 -1.62535481  0.38578904
   0.66376682 -1.41970566]
 [ 0.96666194 -0.68051125  0.86419713 -2.02803309  0.70481046  0.54318426
  -0.90122704 -1.05106376]
 [-1.03823242 -0.30125604  0.29369403  0.34110531 -0.25871319 -0.5525009
   1.08410506  0.18975028]
 [-0.31047456  0.69567149 -2.47769702 -0.7553853  -1.36809576  0.39458803
  -0.71212114 -0.58026815]]
V
 (4, 8)


Here Q is query"What I am looking for"
Here K is key "What I can actually offer"
Here V is value "What I am actually offering"

Here we have four words my name is fahad so for "My" we have one key,query and value similarly for others 

\text{self attention} = softmax\bigg(\frac{Q.K^T}{\sqrt{d_k}}+M\bigg)


In [3]:
np.matmul(q, k.T)#here we ar emultiplying the query vector with the key vector

array([[-4.32470117,  1.54731559,  0.4511383 , -4.0878219 ],
       [ 2.87296613, -0.40901695,  0.11229128, -0.39090134],
       [ 4.08082312,  0.53211112, -0.793667  ,  1.22980007],
       [-0.18655167, -0.05066708,  0.83554072, -0.58491404]])

In [4]:
# Why we need sqrt(d_k) in denominator because multiplication "np.matmul(q, k.T)" has variance 
q.var(), k.var(), np.matmul(q, k.T).var()

(0.9289536360526895, 0.8258056671399958, 4.169402038505689)

In [5]:
scaled = np.matmul(q, k.T) / math.sqrt(d_k)
q.var(), k.var(), scaled.var()

(0.9289536360526895, 0.8258056671399958, 0.521175254813211)

Now we can see variance has decreased

In [6]:
scaled


array([[-1.52901276,  0.54705867,  0.15950148, -1.44526329],
       [ 1.01574691, -0.14460933,  0.03970096, -0.13820449],
       [ 1.44278885,  0.18812969, -0.28060366,  0.43479998],
       [-0.06595598, -0.01791352,  0.29540826, -0.20679834]])


Masking
This is to ensure words don't get context from words generated in the future.
Not required in the encoders, but required in the decoders
Because in encoders we push all corpus at one to bring out some contxt out of it. 

In [9]:
mask = np.tril(np.ones( (L, L) ))
mask

array([[1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 1.]])

In [10]:
mask[mask == 0] = -np.infty
mask[mask == 1] = 0
scaled + mask

array([[-1.52901276,        -inf,        -inf,        -inf],
       [ 1.01574691, -0.14460933,        -inf,        -inf],
       [ 1.44278885,  0.18812969, -0.28060366,        -inf],
       [-0.06595598, -0.01791352,  0.29540826, -0.20679834]])

In [11]:
def softmax(x):
  #Softmax fuction is for probability distribution
  return (np.exp(x).T / np.sum(np.exp(x), axis=-1)).T


In [12]:
attention = softmax(scaled + mask)
attention

array([[1.        , 0.        , 0.        , 0.        ],
       [0.76139744, 0.23860256, 0.        , 0.        ],
       [0.68323151, 0.1948392 , 0.12192929, 0.        ],
       [0.22971989, 0.2410256 , 0.32971378, 0.19954072]])

In [13]:
new_v = np.matmul(attention, v)
new_v

array([[ 1.93841437,  0.66362347,  0.56425023, -1.09439297, -0.03839097,
        -0.68948602, -1.22307951,  1.56018408],
       [ 1.06557988,  0.19574219,  0.54004458, -0.92956531, -0.01056279,
        -0.39328143, -0.61874831,  0.99305028],
       [ 1.24090607, -0.1198517 ,  0.6076907 , -0.85118665,  0.09242065,
        -0.37267373, -0.41744399,  1.04859547],
       [ 0.55792807, -1.20551915,  0.38610887, -0.30385878,  0.38441137,
        -0.03254864,  0.42653801,  0.37863043]])

In [14]:
v

array([[ 1.93841437,  0.66362347,  0.56425023, -1.09439297, -0.03839097,
        -0.68948602, -1.22307951,  1.56018408],
       [-1.71969595, -1.29729969,  0.46280264, -0.40358873,  0.07823885,
         0.55192812,  1.30971481, -0.8167133 ],
       [ 2.06336379, -2.62853452,  1.08263627, -0.20362604,  0.84808643,
        -0.07489903,  1.33699368,  1.16261082],
       [-0.76772795, -0.8951565 , -1.06253005,  0.56108139,  0.47482632,
         0.08773371, -0.24554556, -0.83318772]])

In [16]:
def softmax(x):
  return (np.exp(x).T / np.sum(np.exp(x), axis=-1)).T

def scaled_dot_product_attention(q, k, v, mask=None):
  d_k = q.shape[-1]
  scaled = np.matmul(q, k.T) / math.sqrt(d_k)
  if mask is not None:
    #For encoders it is none
    scaled = scaled + mask
  attention = softmax(scaled)
  out = np.matmul(attention, v)
  return out, attention

In [19]:
values, attention = scaled_dot_product_attention(q, k, v, mask=mask)
print("Q\n", q)
print("K\n", k)
print("V\n", v)
print("New V\n", values)
print("Attention\n", attention)

Q
 [[ 1.09055468 -2.0516344   0.91705551 -0.80064762 -0.70158921 -0.8760401
  -0.1307713   2.34696991]
 [ 0.43011261  0.45461376  0.45879596  0.54999023 -0.53638167 -0.25319405
   0.24376715 -0.89314806]
 [ 0.45241956  2.64528065  0.50091089 -0.66364911 -0.19422367  0.72443613
   0.95165321 -0.68233126]
 [-0.65959933 -1.69094675  0.14901582  0.56326842 -0.45650084  0.2401893
  -0.5038496  -0.17789481]]
K
 [[-0.12870307  0.76705569  0.34871384  0.39214776 -1.62535481  0.38578904
   0.66376682 -1.41970566]
 [ 0.96666194 -0.68051125  0.86419713 -2.02803309  0.70481046  0.54318426
  -0.90122704 -1.05106376]
 [-1.03823242 -0.30125604  0.29369403  0.34110531 -0.25871319 -0.5525009
   1.08410506  0.18975028]
 [-0.31047456  0.69567149 -2.47769702 -0.7553853  -1.36809576  0.39458803
  -0.71212114 -0.58026815]]
V
 [[ 1.93841437  0.66362347  0.56425023 -1.09439297 -0.03839097 -0.68948602
  -1.22307951  1.56018408]
 [-1.71969595 -1.29729969  0.46280264 -0.40358873  0.07823885  0.55192812
   1.3097